<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# Validation and cross-validation

In this exercise you will implement a validation pipeline.

At the end of the MSLE exercise you tested your model against the training and test datasets. As you should observe, there's a gap between the results. By validating your model, not only should you be able to anticipate the test time performance, but also have a method to compare different models.

Implement the basic validation method, i.e. a random split. Test it with your model from Exercise MSLE.

In [1]:
%matplotlib inline

!wget -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
!wget -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1

--2023-10-26 14:43:50--  https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/zey0gx91pna8irj/mieszkania.csv [following]
--2023-10-26 14:43:51--  https://www.dropbox.com/s/dl/zey0gx91pna8irj/mieszkania.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3e4390aab5f5133bbf7a69a7c2.dl.dropboxusercontent.com/cd/0/get/CGU4zr5W253GsKqIPs0BEKOT5j2FLc3XHqieQVBg7nxuomJlZU2xMTMoR8cx8KUZZQV-RWIScCO0H540ToSYwdXr-vwc5YUDtSx7IjnHpZ_C0Hg0zrUEknShmNfFJmpTHWs/file?dl=1# [following]
--2023-10-26 14:43:52--  https://uc3e4390aab5f5133bbf7a69a7c2.dl.dropboxusercontent.com/cd/0/get/CGU4zr5W253GsKqIPs0BEKOT5j2FLc3XHqieQVBg7nxuomJlZU2xMTMoR8cx8KUZZQV-RWIScCO0H540ToSYwdXr-vwc5YUDtSx7IjnHpZ_C0Hg0zrU

In [7]:
import numpy as np
import pandas as pd
from typing import Tuple

In [6]:
def load2(name: str) -> Tuple[np.ndarray, np.array, np.ndarray]:
    data = pd.read_csv(name)
    x = data.loc[:, ((data.columns != 'cena') & (data.columns != 'dzielnica'))]
    dum = data['dzielnica']
    dum = pd.get_dummies(dum)
    x = pd.concat([x, dum], axis=1)
    y = data['cena']

    return x.to_numpy().astype(float), y.to_numpy().astype(float)

In [62]:
x_train, y_train = load2('mieszkania.csv')
x_test, y_test = load2('mieszkania_test.csv')

In [63]:
class MSLEModel:
    def __init__(self):
        self.weights = []
        self.bias = 0

    def msle(self, ys, ps):
      assert len(ys) == len(ps)
      return np.mean((np.log(1 + ys) - np.log(1 + ps)) ** 2)

    def predict(self, xs):
      return xs @ self.weights + self.bias

    def evaluate(self, xs, ys):
        ps = self.predict(self.weights, self.bias, xs)
        return self.msle(ys, ps)

    def fit(self, xs, ys):
        n_epochs = 400000
        step_size = 1e4
        b, f = xs.shape
        self.weights = np.random.uniform(low=-1 / np.sqrt(f), high=1 / np.sqrt(f), size=[f])
        self.bias = (np.exp(np.mean(np.log(1 + ys))) - 1)
        for i in range(n_epochs):
          prediction = self.predict(xs)
          grad_help = 2 * (np.log(prediction + 1) - np.log(ys + 1)) / (1 + prediction)
          grad_bias = np.mean(grad_help)
          grad_weights = (1 / b) * xs.T @ grad_help
          self.weights -= step_size * grad_weights
          self.bias -= step_size * grad_bias


In [64]:
#######################################################
# TODO: Implement the basic validation method,        #
# compare MSLE on training, validation, and test sets #
#######################################################
from sklearn.model_selection import train_test_split
x_train = x_train.astype(np.float64)
x_test = x_test.astype(np.float64)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=42)

model = MSLEModel()
model.fit(x_train, y_train)

y_train_pred = model.predict(x_train)
y_val_pred = model.predict(x_val)
y_test_pred = model.predict(x_test)

# Calculate MSLE for each set.
train_msle = model.msle(y_train, y_train_pred)
val_msle = model.msle(y_val, y_val_pred)
test_msle = model.msle(y_test, y_test_pred)
print(f"MSLE on training set: {train_msle}")
print(f"MSLE on validation set: {val_msle}")
print(f"MSLE on test set: {test_msle}")

MSLE on training set: 0.050334035793693506
MSLE on validation set: 0.05931517615205444
MSLE on test set: 0.07810061688825645


To make the random split validation reliable, a huge chunk of training data may be needed. To get over this problem, one may apply cross-validaiton.

![alt-text](https://chrisjmccormick.files.wordpress.com/2013/07/10_fold_cv.png)

Let's now implement the method. Make sure that:
* number of partitions is a parameter,
* the method is not limited to `mieszkania.csv`,
* the method is not limited to one specific model.

In [ ]:
####################################
# TODO: Implement cross-validation #
####################################
def cross_validation(partitions, x_train, y_train, model):
    samples = len(x_train)
    partition_size = samples // partitions
    results = np.zeros(partitions)

    for i in range(partitions):
        start = i * partition_size
        end = samples if i == partitions - 1 else (i + 1) * partition_size
        x_train_part = np.concatenate((x_train[:start], x_train[end:]), axis=0)
        y_train_part = np.concatenate((y_train[:start], y_train[end:]))
        x_val_part = x_train[start:end]
        y_val_part = y_train[start:end]

        model.fit(x_train_part, y_train_part)
        y_val_pred = model.predict(x_val_part)

        error = model.msle(y_val_part, y_val_pred)
        results[i] = error

    return results.mean()


model = MSLEModel()
print(cross_validation(2, x_train, y_train, model))
print(cross_validation(5, x_train, y_train, model))
print(cross_validation(10, x_train, y_train, model))

0.053054070360327554


Recall that sometimes validation may be tricky, e.g. significant class imbalance, having a small number of subjects, geographically clustered instances...

What could in theory go wrong here with random, unstratified partitions? Think about potential solutions and investigate the data in order to check whether these problems arise here.

In [ ]:
##############################
# TODO: Investigate the data #
##############################